In [2]:
import pandas as pd
import json
import sqlite3

pd.set_option("display.max_columns", None)



In [4]:
ORDERS_PATH = r"C:\Users\chenn\OneDrive\B tech\orders.csv"
USERS_PATH = r"C:\Users\chenn\OneDrive\B tech\users.json"
RESTAURANTS_SQL_PATH = r"C:\Users\chenn\OneDrive\B tech\restaurants.sql"

In [5]:
orders = pd.read_csv(ORDERS_PATH)

print("Orders Data")
display(orders.head())
print("Orders Shape:", orders.shape)


Orders Data


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


Orders Shape: (10000, 6)


In [6]:
with open(USERS_PATH, "r") as f:
    users_data = json.load(f)

users = pd.json_normalize(users_data)

print("Users Data")
display(users.head())
print("Users Shape:", users.shape)


Users Data


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


Users Shape: (3000, 4)


In [7]:
conn = sqlite3.connect(":memory:")


In [8]:
with open(RESTAURANTS_SQL_PATH, "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

print("✅ restaurants.sql executed successfully")


✅ restaurants.sql executed successfully


In [9]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)



,name
0,restaurants


In [10]:
restaurants = pd.read_sql(
    "SELECT * FROM restaurants",
    conn
)

print("Restaurants Data")
display(restaurants.head())
print("Restaurants Shape:", restaurants.shape)


Restaurants Data


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


Restaurants Shape: (500, 4)


In [11]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [12]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left",
    suffixes=("_user", "_restaurant")
)

print("Final Dataset")
display(final_df.head())
print("Final Shape:", final_df.shape)


Final Dataset


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_user,name,city,membership,restaurant_name_restaurant,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


Final Shape: (10000, 12)


In [15]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])

if "city_user" in final_df.columns:
    final_df.rename(columns={"city_user": "user_city"}, inplace=True)

if "city_restaurant" in final_df.columns:
    final_df.rename(columns={"city_restaurant": "restaurant_city"}, inplace=True)

final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   order_id                    10000 non-null  int64         
 1   user_id                     10000 non-null  int64         
 2   restaurant_id               10000 non-null  int64         
 3   order_date                  10000 non-null  datetime64[ns]
 4   total_amount                10000 non-null  float64       
 5   restaurant_name_user        10000 non-null  object        
 6   name                        10000 non-null  object        
 7   city                        10000 non-null  object        
 8   membership                  10000 non-null  object        
 9   restaurant_name_restaurant  10000 non-null  object        
 10  cuisine                     10000 non-null  object        
 11  rating                      10000 non-null  float64    

In [19]:
print("ORDERS columns:", orders.columns.tolist())
print("USERS columns:", users.columns.tolist())
print("RESTAURANTS columns:", restaurants.columns.tolist())
print("FINAL columns:", final_df.columns.tolist())


ORDERS columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']
USERS columns: ['user_id', 'name', 'city', 'membership']
RESTAURANTS columns: ['restaurant_id', 'restaurant_name', 'cuisine', 'rating']
FINAL columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_user', 'name', 'city', 'membership', 'restaurant_name_restaurant', 'cuisine', 'rating']


In [22]:
final_df.to_csv(
    r"C:\Users\chenn\OneDrive\B tech\final_food_delivery_dataset.csv",
    index=False
)
